<h2>Project #2 - Web Scraping Billboard And Spotify API</h2>
Sergey Khegay

### Import necessary Libraries

In [1]:
# importing the libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data

### Define Functions and Lists for Web Scarping

Define function to web scrape **artist name's** from given Billboard link and store it in **Artists** list.

In [2]:
Artists = []
def webscraping_name(link_url,list_name):
    url_content = requests.get(link_url)
    soup = BeautifulSoup(url_content.content, 'html.parser')
    name_list = soup.find_all('span', class_="chart-element__information__artist")
    for i in name_list:
        list_name.append(i.get_text())
    print('Artist name\'s  web scraped.')

Define function to web scrape **name of the songs**  from given Billboard link and store it in **Songs** list.

In [3]:
Songs = []
def webscraping_songs(link_url,list_name):
    url_content = requests.get(link_url)
    soup = BeautifulSoup(url_content.content, 'html.parser')
    name_list = soup.find_all('span', class_="chart-element__information__song")
    for i in name_list:
        list_name.append(i.get_text())
    print('Track name\'s  web scraped.')

### Run Web Scraping Functions and Create Dataframe

Define **Billboard URL** and run the funtions.

In [4]:
url = "https://www.billboard.com/charts/billboard-global-200"
webscraping_name(url,Artists)
webscraping_songs(url,Songs)

Artist name's  web scraped.
Track name's  web scraped.


Create **Billboard dataframe** and assign lists to relative columns.

In [5]:
df_billboard = pd.DataFrame()
df_billboard['Song'] = Songs
df_billboard['Artist'] = Artists
#df_billboard

### Define Spotify Credentials and Access

In [6]:
client_id = '2ebbbedc6bd84d989c84ca1222925180'#{spotify client id}
client_secret = '8740693f8ff8484b9218408e79620cca'#{spotify secret id}
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

### Define Functions and Lists to Return Information Using API

Define function to get **track uri** from Spotify request and store data in the **track_uris** list.

In [7]:
#Track uri List
track_uris = []

In [8]:
def get_track_uris(song_list,artist_list,uri_list):
    for i in range(0, len(song_list)):
        song = str(artist_list[i])+','+str(song_list[i])
        search = sp.search(song)
        if search['tracks']['total']==0:
            song = str(song_list[i])
            search = sp.search(song)
        track_uri = search['tracks']['items'][0]['uri']
        uri_list.append(track_uri)
    print('Track uris has been added to list.')

Define function to request **audio features** and store it in relative lists using **track uri**.

In [9]:
#Audio Features List
acousticness = []
danceability = []
energy = []
instrumentalness = []
liveness = []
loudness = []
speechiness = []
valence = []
tempo = []

In [10]:
def audio_features(uri_list):
    for i in uri_list:
        features = sp.audio_features(i)
        acousticness.append(features[0]['acousticness'])
        danceability.append(features[0]['danceability'])
        energy.append(features[0]['energy'])
        instrumentalness.append(features[0]['instrumentalness'])
        liveness.append(features[0]['liveness'])
        loudness.append(features[0]['loudness'])
        speechiness.append(features[0]['speechiness'])
        valence.append(features[0]['tempo'])
        tempo.append(features[0]['valence'])
    print('Audio features extracted.')

Define function to request additional information including **album, track popularity, release data**, and whether the track is **explicit**. 

In [11]:
#Track Info List
popularity = []
album_name = []
popularity = []
explicit = []
release_date = []

In [12]:
#Additional Track Information
def track_information(uri_list):
    for i in uri_list:
        track_info = sp.track(i)
        popularity.append(track_info['popularity'])
        album_name.append(track_info['album']['name'])
        explicit.append(track_info['explicit'])
        release_date.append(track_info['album']['release_date'])
    print('Additional track information extracted.')

### Run API Functions Get Track Infromation From Spotify and Create Dataframe
Request all information using Spotify API request.

In [13]:
get_track_uris(Songs,Artists,track_uris)
track_information(track_uris)
audio_features(track_uris) 

Loop #: 5
Elapsed Time: 4.316382884979248 seconds
Loop #: 10
Elapsed Time: 9.500347137451172 seconds
Loop #: 15
Elapsed Time: 14.285942792892456 seconds
Loop #: 20
Elapsed Time: 16.99335217475891 seconds
Loop #: 25
Elapsed Time: 22.15664553642273 seconds
Loop #: 30
Elapsed Time: 26.482297658920288 seconds
Loop #: 35
Elapsed Time: 30.601987600326538 seconds
Loop #: 40
Elapsed Time: 33.417479276657104 seconds
Loop #: 45
Elapsed Time: 38.43265390396118 seconds
Loop #: 50
Elapsed Time: 40.962451696395874 seconds
Loop #: 55
Elapsed Time: 46.13368821144104 seconds
Loop #: 60
Elapsed Time: 50.95736789703369 seconds
Loop #: 65
Elapsed Time: 54.675660133361816 seconds
Loop #: 70
Elapsed Time: 58.80479574203491 seconds
Loop #: 75
Elapsed Time: 61.757699966430664 seconds
Loop #: 80
Elapsed Time: 65.3335599899292 seconds
Loop #: 85
Elapsed Time: 70.31434965133667 seconds
Loop #: 90
Elapsed Time: 73.7074522972107 seconds
Loop #: 95
Elapsed Time: 78.33635783195496 seconds
Loop #: 100
Elapsed Time: 8

Create **Spotify dataframe** and assign lists to relative columns.

In [14]:
df_spotify = pd.DataFrame()
df_spotify['Track_URI'] = track_uris
df_spotify['Explicit'] = explicit
df_spotify['Album Name'] = album_name
df_spotify['Release Date'] = release_date
df_spotify['Popularity'] = popularity
df_spotify['Acousticness'] = acousticness
df_spotify['Danceability'] = danceability
df_spotify['Energy'] = energy
df_spotify['Instrumentalness'] = instrumentalness
df_spotify['Liveness'] = liveness
df_spotify['Loudness'] = loudness
df_spotify['Speechiness'] = speechiness
df_spotify['Valence'] = valence
df_spotify['Tempo'] = tempo

### Combine Dataframes, Produce Descriptive Statisic ,and Export Dataframe to CSV

Combine **Billboard** and **Spotify** Dataframes.

In [15]:
final_df = pd.concat([df_billboard, df_spotify], axis=1, sort=False)
#final_df

In [16]:
final_df

,Song,Artist,Track_URI,Explicit,Album Name,Release Date,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Valence,Tempo
0,Life Goes On,BTS,spotify:track:249gnXrbfmV8NG6jTEMSwD,False,BE,2020-11-20,95,0.006910,0.566,0.716,0.000000,0.3700,-5.733,0.0424,81.068,0.4500
1,Dakiti,Bad Bunny & Jhay Cortez,spotify:track:74sMXgFr375v5R2h2ce4fr,False,"The Impossible Game Soundtrack, Vol. 50",2020-12-02,2,0.095000,0.465,0.553,0.903000,0.5360,-9.859,0.0313,110.081,0.0622
2,Dynamite,BTS,spotify:track:0t1kP63rueHleOhQkYSXFY,False,Dynamite (DayTime Version),2020-08-28,94,0.011200,0.746,0.765,0.000000,0.0936,-4.410,0.0993,114.044,0.7370
3,Monster,Shawn Mendes & Justin Bieber,spotify:track:2Z8yfpFX0ZMavHkcIeHiO1,False,Monster,2020-11-20,93,0.067600,0.652,0.383,0.000000,0.0828,-7.076,0.0516,145.765,0.5490
4,Mood,24kGoldn Featuring iann dior,spotify:track:3tjFYV6RSFtuktYl3ZtYcq,True,Mood (feat. iann dior),2020-07-24,98,0.221000,0.700,0.722,0.000000,0.2720,-3.558,0.0369,90.989,0.7560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Ocean Eyes,Billie Eilish,spotify:track:7hDVYcQq6MxkdJGweuCtl9,False,dont smile at me,2017-12-22,82,0.816000,0.511,0.363,0.031700,0.0840,-7.650,0.0410,144.892,0.1690
196,Head Shoulders Knees & Toes,Ofenbach & Quarterhead Featuring Norma Jean Ma...,spotify:track:14ngWWxvUSnIMXgF6rzSk1,False,Head Shoulders Knees & Toes (feat. Norma Jean ...,2020-05-08,79,0.024700,0.798,0.828,0.000003,0.1210,-5.360,0.1930,124.962,0.6000
197,Don't Stop Believin',Journey,spotify:track:4bHsxqR3GMrXTxEPLuK5ue,False,Escape,1981,80,0.127000,0.500,0.748,0.000000,0.4470,-9.072,0.0363,118.852,0.5140
198,Freaky Girls,Megan Thee Stallion Featuring SZA,spotify:track:4RAEVQeHanpXyK2lrEUkMk,True,Good News,2020-11-20,72,0.000794,0.821,0.681,0.000019,0.1100,-6.834,0.1770,93.991,0.2490


Using **.describe()** function produce descriptive Statistics.

In [17]:
final_df.describe()

,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Valence,Tempo
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.00000,200.000000
mean,79.575000,0.271020,0.682530,0.613408,0.044412,0.161205,-6.777975,0.110180,122.72472,0.532846
std,21.089052,0.280089,0.151387,0.176597,0.184890,0.125981,3.189632,0.097449,29.04182,0.228447
min,0.000000,0.000387,0.240000,0.037700,0.000000,0.031400,-21.426000,0.023200,67.19600,0.059200
25%,81.000000,0.041900,0.587250,0.517500,0.000000,0.090100,-7.783000,0.041375,98.78225,0.349000
50%,86.000000,0.176500,0.716000,0.643000,0.000000,0.113000,-6.148000,0.060750,121.91450,0.551000
75%,89.000000,0.420500,0.794000,0.742250,0.000048,0.177250,-4.771750,0.152250,142.05125,0.705500
max,98.000000,0.994000,0.957000,0.973000,0.965000,0.907000,-1.746000,0.481000,203.86300,0.963000


Store Results as **billboard.csv** file.

In [18]:
final_df.to_csv('billboard.csv')